In [1]:
import pandas as pd
import datasets
from bs4 import BeautifulSoup
from io import StringIO
import csv
import torch
import difflib
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline,
)


from tqdm import tqdm
import pandas as pd
import re

data = pd.read_csv('../data/pseudo_json_dataset_V3_2048.csv')

/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = 'RUCKBReasoning/TableLLM-7b'
# model_id = 'Open-Orca/Mistral-7B-OpenOrca'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16,
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.98s/it]


In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [6]:
generation_config = GenerationConfig.from_pretrained(model_id)
print(generation_config)


generation_config.max_new_tokens = 256
generation_config.temperature = 0.0001
generation_config.do_sample = True
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



In [7]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    # streamer=streamer,
)


In [8]:
def extract(ip):
    # Find the index of "[/INST]"
    index = ip.find("[/INST]")

    # Extract the value after "[/INST]"
    if index != -1:
        extracted_value = ip[index + len("[/INST]") + 1:].strip()  # Add 1 to skip the "]" character
        return extracted_value
    else:
        print("No value found after '[/INST]'")

def extract_second(ip):
   # Find the first occurrence of "[/INST]"
    first_inst_index = ip.find("[/INST]")

    # Find the second occurrence of "[/INST]" starting from the position after the first one
    second_inst_index = ip.find("[/INST]", first_inst_index + len("[/INST]"))

    # Extract the value after the second "[/INST]"
    if second_inst_index != -1:
        extracted_value = ip[second_inst_index + len("[/INST]") + 1:].strip()  # Add 1 to skip the "]" character
        return extracted_value
    else:
        print("Two occurrences of '[/INST]' not found")


In [9]:
output_df = pd.DataFrame(columns=['index', 'prompt', 'answer', 'prediction'])

In [10]:
def prompt_formatter(csv_data, question):
    ip = f"""[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    {csv_data}

    ### [Question]
    {question}

    ### [Solution]
    The solution is mostly numerical example: 1234 or 890 or 45. Do not elaborate. Be exact and precise. 
    [/INST]"""
    return ip 

In [11]:
# index = 20 
# soup = BeautifulSoup(data.loc[index, 'html_content'], 'html.parser')

# # Find the table
# table = soup.find('table')
# # Open StringIO object for writing
# csv_buffer = StringIO()


# # Write rows to the StringIO object
# writer = csv.writer(csv_buffer)
# trows = table.find_all('tr')
# for trow in trows:
#     writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


# # Get the CSV data as a string
# csv_data = csv_buffer.getvalue()

# ip = prompt_formatter(csv_data, data.loc[10, 'question'])

# output = llm(ip)[0]['generated_text']
# print(data.loc[index, 'answer'])

# pred = extract(output) 
# print(pred)
# print('***')

In [12]:
def twoshot_prompt_formatter(csv_data, question):
    ip = f"""[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question]. Be specific and concise. 

    ### [Table]
    1,-,2002,2001,2000
    2,net sales,$ 5742,$ 5363,$ 7983
    3,cost of sales,4139,4128,5817
    4,gross margin,$ 1603,$ 1235,$ 2166
    5,gross margin percentage,28% ( 28 % ),23% ( 23 % ),27% ( 27 % )


    ### [Question]
    what is the gross margin in 2002?

    ### [Solution]
    The solution is mostly numerical example: 1234 or 890 or 45. Do not elaborate. Be exact and precise. 

    [/INST] Answer: $1603.

    [INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question]. Be specific and concise.

    ### [Table]
    {csv_data}

    ### [Question]
    {question}

    ### [Solution]
    The solution is mostly numerical example: 1234 or 890 or 45. Do not elaborate. Be exact and precise. 
    [/INST] Answer: """
    return ip 

In [13]:
# soup = BeautifulSoup(data.loc[10, 'html_content'], 'html.parser')

# # Find the table
# table = soup.find('table')
# # Open StringIO object for writing
# csv_buffer = StringIO()


# # Write rows to the StringIO object
# writer = csv.writer(csv_buffer)
# trows = table.find_all('tr')
# for trow in trows:
#     writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


# # Get the CSV data as a string
# csv_data = csv_buffer.getvalue()

# ip = twoshot_prompt_formatter(csv_data, data.loc[10, 'question'])

# output = llm(ip)[0]['generated_text']
# print(data.loc[10, 'answer'])

# pred = extract_second(output) 
# print(pred)
# print('***')

In [14]:
# example= """    " 1"," years ended december 31,"," 2016"," 2015"," 2014"
# " 2"," aes corporation"," $ -50 ( 50 )"," $ -31 ( 31 )"," $ -34 ( 34 )"
# " 3"," chile"," -9 ( 9 )"," -18 ( 18 )"," -30 ( 30 )"
# " 4"," colombia"," -8 ( 8 )"," 29"," 17"
# " 5"," mexico"," -8 ( 8 )"," -6 ( 6 )"," -14 ( 14 )"
# " 6"," philippines"," 12"," 8"," 11"
#  7 , united kingdom , $ 533155 , $ 469127 , 445052 
# " 7"," united kingdom"," 13"," 11"," 12"
# " 8"," argentina"," 37"," 124"," 66"
# " 9"," other"," -2 ( 2 )"," -10 ( 10 )"," -17 ( 17 )"
# " 10"," total ( 1 )"," $ -15 ( 15 )"," $ 107"," $ 11""""
# quest = """What is the value for united kingdom ?"""
# answer = """['13' '11' '12' '$ 533155' '$ 469127' '445052']"""

In [15]:
def convert_to_csv(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    # Write rows to the StringIO object
    writer = csv.writer(csv_buffer)
    trows = table.find_all('tr')
    final_csv = ''
    for trow in trows:
        writer = csv.writer(csv_buffer)
        writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
        csv_data = csv_buffer.getvalue().replace('\n', '').replace('  ', '')
        csv_buffer = StringIO()
        final_csv += csv_data + '\n'
    return final_csv

In [16]:
# with open(filename, 'w') as file:
for i, row in tqdm(data[15000:len(data)].iterrows()):
    # soup = BeautifulSoup(row['html_content'], 'html.parser')

    # # Find the table
    # table = soup.find('table')
    # # Open StringIO object for writing
    # csv_buffer = StringIO()


    # # Write rows to the StringIO object
    # writer = csv.writer(csv_buffer)
    # trows = table.find_all('tr')
    # for trow in trows:
    #     writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # # Get the CSV data as a string
    # csv_data = csv_buffer.getvalue()
    
    csv_data = convert_to_csv(row['html_content'])

    ip = prompt_formatter(csv_data, row['question'])

    output = llm(ip)[0]['generated_text']
    print(i)
    print(ip)
    print('----------')
    print('answer', row['answer'])

    pred = extract(output) 
    print('predictions', pred)
    print('***')
    output_df.at[i, 'index'] = i
    output_df.at[i, 'prompt'] = ip
    output_df.at[i, 'answer'] = data.loc[i,'answer']
    output_df.at[i, 'prediction'] = pred

0it [00:00, ?it/s]

1it [00:47, 47.43s/it]

15000
[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    " 1"," -"," 2013"," 2012"," 2011"
" 2"," average risk-free interest rate"," 0.8% ( 0.8 % )"," 0.7% ( 0.7 % )"," 1.7% ( 1.7 % )"
" 3"," expected dividend yield"," none"," none"," none"
" 4"," expected volatility"," 31% ( 31 % )"," 31% ( 31 % )"," 27% ( 27 % )"
" 5"," expected life ( years )"," 4.6"," 4.6"," 4.5"
" 6"," fair value per share"," $ 19.47"," $ 23.93"," $ 22.78"


    ### [Question]
    What is expected volatility for 2013 ?

    ### [Solution]
    The solution is mostly numerical example: 1234 or 890 or 45. Do not elaborate. Be exact and precise. 
    [/INST]
----------
answer 31% ( 31 % )
predictions The expected volatility for 2013 is 31%.
***


2it [09:24, 323.67s/it]

15001
[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    " 1"," calendar year:"," pre-tax catastrophe losses"
" 2"," ( dollars in millions )"," -"
" 3"," 2014"," $ 62.2"
" 4"," 2013"," 195.0"
" 5"," 2012"," 410.0"
" 6"," 2011"," 1300.4"
" 7"," 2010"," 571.1"


    ### [Question]
    What is ( dollars in millions ) for pre-tax catastrophe losses ?

    ### [Solution]
    The solution is mostly numerical example: 1234 or 890 or 45. Do not elaborate. Be exact and precise. 
    [/INST]
----------
answer -
predictions he table provides information on the pre-tax catastrophe losses for different years. The "pre-tax catastrophe losses" column shows the amount of losses before accounting for taxes. The values in this column are presented in millions of dollars.

To answer the question, the pre-tax catastrophe losses for each year are as follows:

- 2014: $62.2 million
- 2013: $195.0 million
- 2012: $410.0 million
-

...

15573
[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    " 1"," 2015"," high"," low"
" 2"," quarter ended march 31"," $ 101.88"," $ 93.21"
" 3"," quarter ended june 30"," 98.64"," 91.99"
" 4"," quarter ended september 30"," 101.54"," 86.83"
" 5"," quarter ended december 31"," 104.12"," 87.23"
" 6"," 2014"," high"," low"
" 7"," quarter ended march 31"," $ 84.90"," $ 78.38"
" 8"," quarter ended june 30"," 90.73"," 80.10"
" 9"," quarter ended september 30"," 99.90"," 89.05"
" 10"," quarter ended december 31"," 106.31"," 90.20"


    ### [Question]
    What is quarter ended december 31 for high ?

    ### [Solution]
    The solution is mostly numerical example: 1234 or 890 or 45. Do not elaborate. Be exact and precise. 
    [/INST]
----------
answer 104.12
predictions he quarter ended December 31 for high is $104.12.
***


575it [16:54:20, 164.14s/it]

15574
[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    " 1"," years ended december 31"," pension benefits 2013"," pension benefits 2012"," pension benefits 2011"," pension benefits 2013"," pension benefits 2012"," 2011"
" 2"," service cost"," $ 682"," $ 555"," $ 619"," $ 102"," $ 82"," $ 110"
" 3"," interest cost"," 665"," 661"," 718"," 107"," 121"," 141"
" 4"," expected return on plan assets"," -1097 ( 1097 )"," -970 ( 970 )"," -972 ( 972 )"," -126 ( 126 )"," -136 ( 136 )"," -142 ( 142 )"
" 5"," net amortization"," 336"," 185"," 201"," -50 ( 50 )"," -35 ( 35 )"," -17 ( 17 )"
" 6"," termination benefits"," 58"," 27"," 59"," 50"," 18"," 29"
" 7"," curtailments"," -23 ( 23 )"," -10 ( 10 )"," -86 ( 86 )"," -11 ( 11 )"," -7 ( 7 )"," 1"
" 8"," settlements"," 23"," 18"," 4"," 2014"," 2014"," 2014"
" 9"," net periodic benefit cost"," $ 644"," $ 466"," $ 543"," $ 72"," $ 43"," $ 122"


    ### [Question]
    What

576it [16:54:59, 126.63s/it]

15575
[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    " 1"," 2011"," $ 74896"
" 2"," 2012"," 625884"
" 3"," 2013"," 618"
" 4"," 2014"," 1750479"
" 5"," 2015"," 600489"
" 6"," thereafter"," 2541858"
" 7"," total cash obligations"," 5594224"
" 8"," unamortized discounts and premiums net"," -6836 ( 6836 )"
" 9"," balance as of december 31 2010"," $ 5587388"


    ### [Question]
    What is 2015 ?

    ### [Solution]
    The solution is mostly numerical example: 1234 or 890 or 45. Do not elaborate. Be exact and precise. 
    [/INST]
----------
answer 600489
predictions 2015 is the year 2015.
***


In [ ]:
output_df.to_csv('results/zero_shot_test.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'results'